In [2]:
""""
Figaro times collection times script. 
"""
import numpy as np
import regex as re
import os
import pandas as pd 
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

path_src = "/local/scratch/Figaro/figaro-code"
exp_names = ["mkl"]
exp_paths = {"figaro": "", "mkl": "comparisons/accuracy/python/numpy"}
txt_name = "error.txt"

numeric_const_pattern = r"Relative error is: ([+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)"
rx = re.compile(numeric_const_pattern)


row_nums = [512, 1024, 2048, 4096]
column_nums = [1, 2, 4, 8, 16, 32, 64, 128, 256]
row_col_nums = []

skip_dbs = {"mkl": [11, 12, 13, 14, 15, 16, 17, 18, 19, 20], "figaro": []}
join_order = "FullJoin"
#skip_dbs = {"mkl": [12, 15, 16, 18, 19, 20], "figaro": []}

for row_num in row_nums:
    for col_num in column_nums:
        row_col_nums.append((row_num, col_num))



df_measurement_exps = {}

for exp_name in exp_names:
    #Initialize paths for the current experiment 
    exp_path = exp_paths[exp_name]
    accur_path = os.path.join(path_src, exp_path)
    df_measurement = pd.DataFrame(columns=column_nums)
    gather_times_path = os.path.join(accur_path, txt_name)


    for db_idx, db_num in enumerate(range(1, 36)): 
        db_name = "DBCartesianProductAccuracy{}".format(db_num)
        if db_num in skip_dbs[exp_name]:
            continue
        path_txt = os.path.join(accur_path, db_name, join_order, txt_name)
        
        # Writng header files
        row_num = row_col_nums[db_idx][0]
        col_num = row_col_nums[db_idx][1]
        with open(path_txt, 'r') as file_accur:
            for line in file_accur:
                print(line)
                match = rx.search(line)
                print(match)
                if match:
                    relative_error = float(match.group(1))

        df_measurement.at[row_num, col_num] = relative_error

    df_measurement = df_measurement.astype(float)
    print(df_measurement)
    df_measurement_exps[exp_name] = df_measurement



Absolute error is: 6.7801306856437325e-09

None
Frobenius norm of comp is: 2955383.883483159

None
Relative error is: 2.2941624347131514e-15

<regex.Match object; span=(0, 41), match='Relative error is: 2.2941624347131514e-15'>
Absolute error is: 1.0464688506242072e-08

None
Frobenius norm of comp is: 4106641.5837294576

None
Relative error is: 2.54823516805149e-15

<regex.Match object; span=(0, 39), match='Relative error is: 2.54823516805149e-15'>
Absolute error is: 9.182583360718805e-09

None
Frobenius norm of comp is: 5955138.459993687

None
Relative error is: 1.5419596743899283e-15

<regex.Match object; span=(0, 41), match='Relative error is: 1.5419596743899283e-15'>
Absolute error is: 1.4619780443612943e-08

None
Frobenius norm of comp is: 8283096.941835224

None
Relative error is: 1.7650138041694521e-15

<regex.Match object; span=(0, 41), match='Relative error is: 1.7650138041694521e-15'>
Absolute error is: 1.943083752377196e-08

None
Frobenius norm of comp is: 11791125.138390483